# Bayes Estimator

### <font color='orange'>Universidad Autónoma de Yucatán</font> _Facultad de Matemáticas_

**Teacher:** Dr. Arturo Espinosa Romero <[eromero@correo.uady.mx](mailto:eromero@correo.uady.mx)>

**Student:** Ing. Dayan Bravo Fraga <[dayan3847@gmail.com](mailto:dayan3847@gmail.com)>

# Practice 5: Implicit Extended Kalman Filter for Ball Tracking

## GitHub: [Practice 5](https://github.com/dayan3847/bayes_estimator/tree/main/practice5-ball_tracking)

In [19]:
import sympy as sp

In [20]:
#@title Estado de sistema
X, Y, Z, Xp, Yp, Zp = sp.symbols('X Y Z \dot{X} \dot{Y} \dot{Z}')
# El estado esta compuesto por las coordenadas reales y las velocidades
XX = sp.Matrix([
    [X],
    [Y],
    [Z],
    [Xp],
    [Yp],
    [Zp],
])

sp.Eq(sp.Symbol('\mathbb{X}'), XX, evaluate=False)

Eq(\mathbb{X}, Matrix([
[      X],
[      Y],
[      Z],
[\dot{X}],
[\dot{Y}],
[\dot{Z}]]))

In [21]:
#@title Matriz A "transitionMatrix"
dt = sp.Symbol('\Delta t')
# Esta seria la matrix de transformacion de estado
AA = sp.Matrix([
    [1, 0, 0, dt, 0, 0],
    [0, 1, 0, 0, dt, 0],
    [0, 0, 1, 0, 0, dt],
    [0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 1],
])
sp.Eq(sp.Symbol('\mathbb{A}'), AA, evaluate=False)

Eq(\mathbb{A}, Matrix([
[1, 0, 0, \Delta t,        0,        0],
[0, 1, 0,        0, \Delta t,        0],
[0, 0, 1,        0,        0, \Delta t],
[0, 0, 0,        1,        0,        0],
[0, 0, 0,        0,        1,        0],
[0, 0, 0,        0,        0,        1]]))

In [22]:
#@title Ecuacion de tranformacion de estado
# el estado en k, es el resultado de multiplicar A por el estado anterior
sp.Eq(XX, sp.MatMul(AA, XX, evaluate=False), evaluate=False)

Eq(Matrix([
[      X],
[      Y],
[      Z],
[\dot{X}],
[\dot{Y}],
[\dot{Z}]]), Matrix([
[1, 0, 0, \Delta t,        0,        0],
[0, 1, 0,        0, \Delta t,        0],
[0, 0, 1,        0,        0, \Delta t],
[0, 0, 0,        1,        0,        0],
[0, 0, 0,        0,        1,        0],
[0, 0, 0,        0,        0,        1]])*Matrix([
[      X],
[      Y],
[      Z],
[\dot{X}],
[\dot{Y}],
[\dot{Z}]]))

In [23]:
#@title "Z" Medicion
x, y, xp, yp, r = sp.symbols('x y \dot{x} \dot{y} r')
# La medicion esta compuesta por las coordenadas de la camara y el tamanno del radio (en metros)
ZZ = sp.Matrix([
    [x],
    [y],
    [xp],
    [yp],
    [r],
])
sp.Eq(sp.Symbol('\mathbb{Z}'), ZZ, evaluate=False)

Eq(\mathbb{Z}, Matrix([
[      x],
[      y],
[\dot{x}],
[\dot{y}],
[      r]]))

In [13]:
# @title Funcion h
R = sp.symbols('R')
h_xz = sp.Matrix([
    [x - X / Z],
    [y - Y / Z],
    [xp - (Xp + x * Zp) / Z],
    [yp - (Yp + y * Zp) / Z],
    [r - R / Z],
])
sp.Eq(sp.Function('h')(sp.Symbol('\mathbb{X,Z}')), h_xz, evaluate=False)

Eq(h(\mathbb{X,Z}), Matrix([
[                         -X/Z + x],
[                         -Y/Z + y],
[\dot{x} - (\dot{X} + \dot{Z}*x)/Z],
[\dot{y} - (\dot{Y} + \dot{Z}*y)/Z],
[                         -R/Z + r]]))

In [16]:
# @title Funcion h
h_x__ = sp.Matrix([
    [X / Z],
    [Y / Z],
    [(Xp + (X / Z) * Zp) / Z],
    [(Yp + (Y / Z) * Zp) / Z],
    [R / Z],
])
sp.Eq(
    sp.Eq(
        sp.Function('h')(sp.Symbol('\mathbb{X,Y}'), evaluate=False),
        h_xz, evaluate=False
    ),
    sp.Symbol('\emptyset'), evaluate=False
)

Eq(Eq(h(\mathbb{X,Y}), Matrix([
[                         -X/Z + x],
[                         -Y/Z + y],
[\dot{x} - (\dot{X} + \dot{Z}*x)/Z],
[\dot{y} - (\dot{Y} + \dot{Z}*y)/Z],
[                         -R/Z + r]])), \emptyset)

In [17]:
# @title Jacobiano de h(X,Z) con respecto a X
HH = h_xz.jacobian(XX)
sp.Eq(sp.Symbol(r'\mathbb{H}'), HH, evaluate=False)

Eq(\mathbb{H}, Matrix([
[-1/Z,    0,                     X/Z**2,    0,    0,    0],
[   0, -1/Z,                     Y/Z**2,    0,    0,    0],
[   0,    0, (\dot{X} + \dot{Z}*x)/Z**2, -1/Z,    0, -x/Z],
[   0,    0, (\dot{Y} + \dot{Z}*y)/Z**2,    0, -1/Z, -y/Z],
[   0,    0,                     R/Z**2,    0,    0,    0]]))

In [18]:
# @title Jacobiano de h(X,Z) con respecto a Z
JJ = h_xz.jacobian(ZZ)
sp.Eq(sp.Symbol(r'\mathbb{J}'), JJ, evaluate=False)

Eq(\mathbb{J}, Matrix([
[         1,          0, 0, 0, 0],
[         0,          1, 0, 0, 0],
[-\dot{Z}/Z,          0, 1, 0, 0],
[         0, -\dot{Z}/Z, 0, 1, 0],
[         0,          0, 0, 0, 1]]))